### DataFrames Divvy-Bikeshare: ###

In this Jupyter-notebook the zip-files from Divvy bikeshare, provided from this website: https://divvy-tripdata.s3.amazonaws.com/index.html, will be used.<br>
You need to download these files and copy them into a folder called 'divvy'<br>
The trips-data will be cleaned and sorted by starttime for every year.<br>
After cleaning, the DataFrames are saved as new, annual csv-files in the 'data'-folder.

|column-no.|column-name|datatype|
|---|---|---|
|0|starttime|datetime64[ns]|
|1|tripduration_in_min|float64|
|2|start_station_name|object|
|3|start_station_lat|float64|
|4|start_station_lon|float64|
|5|end_station_name|object|
|6|end_station_lat|float64|
|7|end_station_lon|float64|
|8|customer|object|
|9|bikeid|int64|
||or||
|9|rideable_type|object|

In [1]:
# Import libraries
import pandas as pd
import os
import zipfile
import fnmatch

# Import libraries for checking for newer zip-files on the website
import requests
import re
from bs4 import BeautifulSoup

# Import for json request of stations
import urllib.request
import json

# Folders for loading and saving data
load_path = "divvy/"
save_path = "data/"
if os.path.isdir(load_path) == False:
    os.mkdir(load_path)
if os.path.isdir(save_path) == False:
    os.mkdir(save_path)

# Don't show warnings
import warnings
warnings.simplefilter("ignore", UserWarning)

### Check if newer zip-files with trips data are available on the website: ###

In [2]:
# Define the URL of the webpage
url = 'https://divvy-tripdata.s3.amazonaws.com'

# Send an HTTP GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = str(BeautifulSoup(response.text))
    # Read file-names to list
    website_files = re.findall(r'<key>(.*?)</key>', soup)
    website_files.remove('index.html')
    
else:
    print(f'Failed to retrieve the webpage. Status code: ' + str(response.status_code))

In [3]:
# Create list of downloaded zip files in the 'divvy' folder
divvy_list = []
for filename in os.listdir(load_path):
    if ("ivvy" in filename) and filename.endswith('zip'):
        divvy_list.append(filename)

# Check if there are new zip-files with trips on the website:
missing_files = []

for website_file in website_files:
    if website_file not in divvy_list:
        missing_files.append(website_file)

if len(missing_files) == 0:
    print("The downloaded zip-files of trips in the 'divvy' directory are COMPLETE!")
elif len(missing_files) == 1:
    print(f"The downloaded zip-files of trips in the 'divvy' directory are NOT complete! File: " + str(missing_files) + " is available for download on the website!")
else:
    print(f"The downloaded zip-files of trips in the 'divvy' directory are NOT complete! Files: " + str(missing_files) + " are available for download on the website!")

The downloaded zip-files of trips in the 'divvy' directory are COMPLETE!


### Get informations about current Divvy stations from JSON feed: ###

In [4]:
# Create json object from divvy stations feed
url = urllib.request.urlopen('https://gbfs.lyft.com/gbfs/2.3/chi/en/station_information.json')
stations = json.load(url)
del url

df_list = list(dict(stations['data'])['stations'])
del stations

# Use method json_normalize to read all elements of the list to a DataFrame
df_test = pd.json_normalize(df_list)
del df_list

# Round longitude and latitude to 6 decimals
df_test['lat'] = round(df_test['lat'], 6)
df_test['lon'] = round(df_test['lon'], 6)

# Rename columns
df_test.rename(columns={"name": "station_name"}, inplace=True)

# Clean-up station names, remove whitespaces
df_test['station_name'] = df_test['station_name'].str.strip()
df_test['station_name'] = df_test['station_name'].str.replace("  ", " ")

# Remove duplicates
df_test = df_test[df_test['station_name'].duplicated(keep='first') == False]
df_test.reset_index(drop=True, inplace=True)

# Sort columns in a final DataFrame
df_stations_new = df_test[['station_name', 'lat', 'lon']]
del df_test
df_stations_new.info(show_counts=True)
df_stations_new

# Save DataFrame as csv-file
df_stations_new.to_csv(save_path + 'stations_current.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1678 entries, 0 to 1677
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   station_name  1678 non-null   object 
 1   lat           1678 non-null   float64
 2   lon           1678 non-null   float64
dtypes: float64(2), object(1)
memory usage: 39.5+ KB


In [5]:
# Show current Divvy stations on a map (works with geopandas 0.12.2)
import geopandas as gpd

gdf_stations = gpd.GeoDataFrame(df_stations_new, geometry=gpd.points_from_xy(df_stations_new.lon, df_stations_new.lat), crs="WGS84")
gdf_stations.explore(min_lat=42.1, min_lon=-87.9, max_lat=41.6, max_lon=-87.5, color='deepskyblue')

### Get informations about old Divvy stations (2013-2017) from csv-files in 'divvy'-folder: ###

In [6]:
for year in range(2013, 2018):

    # Create list of stations csv-files of the specified year
    station_list = []
    for filename in os.listdir(load_path):
        if (("ivvy_Stations" in filename) and not ("rips" in filename)) and (str(year) in filename) and filename.endswith('csv'):
            station_list.append(filename)

    # Set dynamically name of the DataFrame, e.g. df_2013.
    station = f'stations_{year}'
    globals()[station] = pd.DataFrame()

    # Create subset of csv file(s) within the zip-files and open them as DataFrame. Concat the single DataFrames into the yearly DataFrame.
    for file in station_list:
        
        # Open the csv-file(s) in a DataFrame
        if (len(station_list) > 1):
            for i in range(0, len(station_list)):
                df_temp = pd.read_csv(load_path + station_list[i], low_memory=False)
                globals()[station] = pd.concat([globals()[station], df_temp])
        else:
            df_temp = pd.read_csv(load_path + station_list[0], low_memory=False)
            globals()[station] = pd.concat([globals()[station], df_temp])
        del df_temp
    
    # Remove duplicates in 'name' and sort to 'id'
    globals()[station].drop_duplicates(subset='name', keep='first', inplace=True)
    globals()[station].sort_values(by='id', inplace=True)

    # Drop unnecessary columns
    drop_col_list = ['city', 'dateCreated', 'landmark', 'online date', 'online_date', 'Unnamed: 7']
    for element in drop_col_list:
        if element in globals()[station].columns:
            globals()[station].drop(columns=[element], inplace=True)
    
    # Rename columns 
    globals()[station].rename(columns = {'name': 'station_name', 'latitude': 'lat', 'longitude': 'lon', 'dpcapacity': 'capacity'}, inplace = True)

    # Clean-up station names, remove whitespaces
    globals()[station]['station_name'] = globals()[station]['station_name'].str.strip()
    globals()[station]['station_name'] = globals()[station]['station_name'].str.replace("  ", " ")

    # Round longitude and latitude to 6 decimals
    globals()[station]['lat'] = round(globals()[station]['lat'], 6)
    globals()[station]['lon'] = round(globals()[station]['lon'], 6)

    # Re-build index
    globals()[station].reset_index(drop=True, inplace=True)

    # Save DataFrame as csv-file
    globals()[station].to_csv(save_path + 'stations_' + str(year) + '.csv', index=False)

### Create csv-files with annual bike-trips: ###

In [7]:
# Create annual csv-files with cleaned bike-trips
# CAUTION: Script runs several minutes, if you select the whole period of time!!
for year in range(2024, 2025):
    
    # Create list of downloaded zip files of the specified year
    divvy_list = []
    for filename in os.listdir(load_path):
        if ("ivvy" in filename) and (str(year) in filename) and filename.endswith('zip'):
            divvy_list.append(filename)
    
    # Set dynamically name of the DataFrame, e.g. df_2013.
    name = f'df_{year}'
    globals()[name] = pd.DataFrame()

    # Create subset of csv file(s) within the zip-files and open them as DataFrame. Concat the single DataFrames into the yearly DataFrame.
    for file in divvy_list:
        zf = zipfile.ZipFile(load_path + file)
        csv_list = []
        # If there is more than 1 relevant trips-file
        if (len(zipfile.ZipFile.namelist(zf)) > 1):
            for i in range(0,len(zipfile.ZipFile.namelist(zf))):
                if ((fnmatch.fnmatch(zipfile.ZipFile.namelist(zf)[i], f'*ivvy*rips*{year}*.csv')) or (fnmatch.fnmatch(zipfile.ZipFile.namelist(zf)[i], f'*{year}*ivvy*ripdata*.csv'))) and not (fnmatch.fnmatch(zipfile.ZipFile.namelist(zf)[i], f'*/*tations*{year}*.csv')) and not (fnmatch.fnmatch(zipfile.ZipFile.namelist(zf)[i], f'*MACOS*')):
                    csv_list.append(zipfile.ZipFile.namelist(zf)[i])
                    #print(zipfile.ZipFile.namelist(zf)[i])
        # If there is only 1 relevant trips-file
        else:
            csv_list.append(zipfile.ZipFile.namelist(zf)[0])
            #print(zipfile.ZipFile.namelist(zf)[i])
        
        # Open the csv-files in a DataFrame
        if (len(csv_list) > 1):
            for i in range(0, len(csv_list)):
                df_temp = pd.read_csv(zf.open(csv_list[i]), low_memory=False)
                # Renaming columns #1
                df_temp.rename(columns = {'01 - Rental Details Rental ID':'trip_id',
                                    '01 - Rental Details Local Start Time':'starttime',
                                    '01 - Rental Details Local End Time':'stoptime',
                                    '01 - Rental Details Bike ID':'bikeid',
                                    '01 - Rental Details Duration In Seconds Uncapped':'tripduration',
                                    '03 - Rental Start Station ID':'from_station_id',
                                    '03 - Rental Start Station Name':'from_station_name',
                                    '02 - Rental End Station ID':'to_station_id',
                                    '02 - Rental End Station Name':'to_station_name',
                                    'User Type':'usertype',
                                    'Member Gender':'gender',
                                    '05 - Member Details Member Birthday Year':'birthyear'}, inplace = True)
                # Renaming columns #2
                df_temp.rename(columns = {'from_station_name': 'start_station_name', 'to_station_name': 'end_station_name', 'usertype': 'customer'}, inplace = True)
                # Renaming columns #3
                df_temp.rename(columns = {'start_time': 'starttime', 'end_time': 'stoptime', 'started_at': 'starttime', 'ended_at': 'stoptime', 'member_casual': 'customer'}, inplace = True)
                globals()[name] = pd.concat([globals()[name], df_temp])
        else:
            df_temp = pd.read_csv(zf.open(csv_list[0]), low_memory=False)
            # Renaming columns #1
            df_temp.rename(columns = {'01 - Rental Details Rental ID':'trip_id',
                                    '01 - Rental Details Local Start Time':'starttime',
                                    '01 - Rental Details Local End Time':'stoptime',
                                    '01 - Rental Details Bike ID':'bikeid',
                                    '01 - Rental Details Duration In Seconds Uncapped':'tripduration',
                                    '03 - Rental Start Station ID':'from_station_id',
                                    '03 - Rental Start Station Name':'from_station_name',
                                    '02 - Rental End Station ID':'to_station_id',
                                    '02 - Rental End Station Name':'to_station_name',
                                    'User Type':'usertype',
                                    'Member Gender':'gender',
                                    '05 - Member Details Member Birthday Year':'birthyear'}, inplace = True)
            # Renaming columns #2
            df_temp.rename(columns = {'from_station_name': 'start_station_name', 'to_station_name': 'end_station_name', 'usertype': 'customer'}, inplace = True)
            # Renaming columns #3
            df_temp.rename(columns = {'start_time': 'starttime', 'end_time': 'stoptime', 'started_at': 'starttime', 'ended_at': 'stoptime', 'member_casual': 'customer'}, inplace = True)
            globals()[name] = pd.concat([globals()[name], df_temp])
        del df_temp

    # Set correct datatypes to columns. The datetime format changed over the years.
    if (year <= 2015):
        globals()[name]['starttime'] = pd.to_datetime(globals()[name]['starttime'], yearfirst=True)
        globals()[name]['stoptime'] = pd.to_datetime(globals()[name]['stoptime'], yearfirst=True)
    elif (year == 2016) or (year == 2017):
        globals()[name]['starttime'] = pd.to_datetime(globals()[name]['starttime'], format='mixed')
        globals()[name]['stoptime'] = pd.to_datetime(globals()[name]['stoptime'], format='mixed')
    else:
        globals()[name]['starttime'] = pd.to_datetime(globals()[name]['starttime'])
        globals()[name]['stoptime'] = pd.to_datetime(globals()[name]['stoptime'])
    if (year == 2018) or (year == 2019):
        globals()[name]['tripduration'] = globals()[name]['tripduration'].astype('string')
        globals()[name]['tripduration'] = globals()[name]['tripduration'].str.replace(".0", "")
        globals()[name]['tripduration'] = globals()[name]['tripduration'].str.replace(",", "")
        globals()[name]['tripduration'] = globals()[name]['tripduration'].astype('int')
    
    # Calculate tripduration in minutes from (stoptime - starttime)
    globals()[name]['tripduration_in_min'] = round(((globals()[name]['stoptime'] - globals()[name]['starttime']) / pd.Timedelta(minutes=1)), 2)

    # Drop values with customer = 'Dependent'
    if ('Dependent' in globals()[name]['customer'].unique()):
        globals()[name].drop(globals()[name][globals()[name]['customer'] == 'Dependent'].index, inplace=True)
    # Drop values with tripduration <= 60 seconds
    if ('tripduration' in globals()[name].columns):
        globals()[name].drop(globals()[name][(globals()[name]['tripduration'] <= 60)].index, inplace=True)
    # Drop values with tripduration_in_min <= 1.0 minutes
    if ('tripduration_in_min' in globals()[name].columns):
        globals()[name].drop(globals()[name][(globals()[name]['tripduration_in_min'] <= 1.0)].index, inplace=True)

    # Substitute values in column 'customer'
    if (year <= 2019):
        globals()[name]['customer'] = globals()[name]['customer'].map({'Subscriber': 'member', 'Customer': 'casual'})

    # Drop unused columns from list
    drop_col_list =['birthday', 'birthyear', 'from_station_id', 'gender', 'stoptime', 'to_station_id', 'trip_id', 'tripduration']
    for element in drop_col_list:
        if element in globals()[name].columns:
            globals()[name].drop(columns=[element], inplace=True)

    # Clean-up station names, remove whitespaces
    globals()[name]['start_station_name'] = globals()[name]['start_station_name'].str.strip()
    globals()[name]['start_station_name'] = globals()[name]['start_station_name'].str.replace("  ", " ")
    globals()[name]['end_station_name'] = globals()[name]['end_station_name'].str.strip()
    globals()[name]['end_station_name'] = globals()[name]['end_station_name'].str.replace("  ", " ")

    # Create complete list of all stations without duplicates:
    df_stations = pd.read_csv(save_path + 'stations_2013.csv')
    for i in range(2014, 2018):
        station = f'stations_{i}.csv'
        df_temp = pd.read_csv(save_path + station)
        df_stations = pd.concat([df_stations, df_temp])
    df_stations = pd.concat([df_stations, df_stations_new])
    del df_temp
    df_stations.drop_duplicates(subset='station_name', keep='first', inplace=True)
    
    # Merging latitude and longitude from stations-DataFrame to trips-DataFrame for years 2013-2019
    if (year <= 2019):
        # Merging with latitude and longitude from stations
        df_merge1 = pd.merge(globals()[name], df_stations, how='left', left_on='start_station_name', right_on='station_name')
        df_merge1.drop(columns=['station_name'], inplace=True)
        df_merge1.rename(columns = {'lat': 'start_station_lat', 'lon': 'start_station_lon'}, inplace = True)
        df_merge2 = pd.merge(df_merge1, df_stations, how='left', left_on='end_station_name', right_on='station_name')
        df_merge2.drop(columns=['station_name'], inplace=True)
        df_merge2.rename(columns = {'lat': 'end_station_lat', 'lon': 'end_station_lon'}, inplace = True)
        del df_merge1
    # Keeping the latitude and longitude values for the years > 2020
    else:
        # Round longitude and latitude to 6 decimals
        globals()[name]['start_station_lat'] = round(globals()[name]['start_lat'], 6)
        globals()[name]['start_station_lon'] = round(globals()[name]['start_lng'], 6)
        globals()[name]['end_station_lat'] = round(globals()[name]['end_lat'], 6)
        globals()[name]['end_station_lon'] = round(globals()[name]['end_lng'], 6)
    del df_stations

    # Re-sorting columns. Differences: 2013-2019 contain 'bikeid', >2020 contain 'rideable_type'
    if (year <= 2019):
        globals()[name] = df_merge2[['starttime', 'tripduration_in_min', 'start_station_name', 'start_station_lat', 'start_station_lon', 'end_station_name', 'end_station_lat', 'end_station_lon', 'customer', 'bikeid']]
        del df_merge2
    else:
        globals()[name] = globals()[name][['starttime', 'tripduration_in_min', 'start_station_name', 'start_station_lat', 'start_station_lon', 'end_station_name', 'end_station_lat', 'end_station_lon', 'customer', 'rideable_type']]

    # Sort according to starttime and rebuild index
    globals()[name].sort_values('starttime', inplace=True)
    globals()[name].reset_index(drop=True, inplace=True)
    
    # Save DataFrame as csv-file
    globals()[name].to_csv(save_path + 'trips_' + str(year) + '.csv')

    # Print when DataFrame for one year has been added
    print(f'Saved trips for {year}: ' + str(globals()[name].shape[0]))

    del globals()[name]

Saved trips for 2024: 140489
